In [55]:
import time
import os
from torch.autograd import Variable
import torchvision
import torch
import random
import numpy as np
import numpy
import networks
# from my_args import  args
# from scipy.misc import imread, imsave
from imageio import imread
from PIL import Image

from AverageMeter import  *
import shutil
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from skimage.measure import compare_ssim

In [2]:
class Args():
    
    def __init__(self):
        self.filter_size = 4
        self.time_step = 0.5
        self.channels = 3
        self.netName = 'DAIN'
        self.use_cuda = True
        
        self.save_which = 1
        self.dtype = torch.cuda.FloatTensor
        
args = Args()

In [3]:
model = networks.__dict__[args.netName](    channel=args.channels,
                                    filter_size = args.filter_size ,
                                    timestep=args.time_step,
                                    training=False)

model = model.eval()

In [4]:
model.eval()

DAIN(
  (initScaleNets_filter): ModuleList(
    (0): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU()
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): MaxPool2d(kern

In [5]:
if args.use_cuda:
    print('USING CUDA')
    model = model.cuda()

USING CUDA


In [6]:
args.SAVED_MODEL = '../../models/dain/best.pth'

In [7]:
if os.path.exists(args.SAVED_MODEL):
    print("The testing model weight is: " + args.SAVED_MODEL)
    if not args.use_cuda:
        pretrained_dict = torch.load(args.SAVED_MODEL, map_location=lambda storage, loc: storage)
        # model.load_state_dict(torch.load(args.SAVED_MODEL, map_location=lambda storage, loc: storage))
    else:
        pretrained_dict = torch.load(args.SAVED_MODEL)
        # model.load_state_dict(torch.load(args.SAVED_MODEL))

    model_dict = model.state_dict()
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    # 4. release the pretrained dict for saving memory
    pretrained_dict = []
else:
    print("*****************************************************************")
    print("**** We don't load any trained weights **************************")
    print("*****************************************************************")

The testing model weight is: ../../models/dain/best.pth


In [8]:
use_cuda=args.use_cuda
save_which=args.save_which
dtype = args.dtype
unique_id =str(random.randint(0, 100000))

In [62]:
def get_padding(intWidth, intHeight, channel):
    
    if intWidth != ((intWidth >> 7) << 7):
        intWidth_pad = (((intWidth >> 7) + 1) << 7)  # more than necessary
        intPaddingLeft =int(( intWidth_pad - intWidth)/2)
        intPaddingRight = intWidth_pad - intWidth - intPaddingLeft
    else:
        intWidth_pad = intWidth
        intPaddingLeft = 32
        intPaddingRight= 32

    if intHeight != ((intHeight >> 7) << 7):
        intHeight_pad = (((intHeight >> 7) + 1) << 7)  # more than necessary
        intPaddingTop = int((intHeight_pad - intHeight) / 2)
        intPaddingBottom = intHeight_pad - intHeight - intPaddingTop
    else:
        intHeight_pad = intHeight
        intPaddingTop = 32
        intPaddingBottom = 32
    
    
    return intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom




def preprocess_image(im):
    X = torch.from_numpy( np.transpose(im, (2,0,1)).astype('float32')/255.0).type(dtype)
    
    intWidth = X.size(2)
    intHeight = X.size(1)
    channel = X.size(0)
    
    
    intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom = get_padding(intWidth, intHeight, channel)
    
    pader = torch.nn.ReplicationPad2d([intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom])
    
    torch.set_grad_enabled(False)
    X = Variable(torch.unsqueeze(X,0))
    X = pader(X)
    
    return X, (intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom)
    

    
    
def interpolate(im1, im2):
    X0, pad_0 = preprocess_image(im1)
    X1, pad_1 = preprocess_image(im2)
    
    y_ = torch.FloatTensor()

    assert pad_0 == pad_1
    
    intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom = pad_0
    
    if use_cuda:
        X0 = X0.cuda()
        X1 = X1.cuda()
        
    y_s,offset,filter = model(torch.stack((X0, X1),dim = 0))
    y_ = y_s[save_which]
    
    
    if use_cuda:
        X0 = X0.data.cpu().numpy()
        if not isinstance(y_, list):
            y_ = y_.data.cpu().numpy()
        else:
            y_ = [item.data.cpu().numpy() for item in y_]
        offset = [offset_i.data.cpu().numpy() for offset_i in offset]
        filter = [filter_i.data.cpu().numpy() for filter_i in filter]  if filter[0] is not None else None
        X1 = X1.data.cpu().numpy()
    else:
        X0 = X0.data.numpy()
        if not isinstance(y_, list):
            y_ = y_.data.numpy()
        else:
            y_ = [item.data.numpy() for item in y_]
        offset = [offset_i.data.numpy() for offset_i in offset]
        filter = [filter_i.data.numpy() for filter_i in filter]
        X1 = X1.data.numpy()
        
    X0 = np.transpose(255.0 * X0.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))


    y_ = [np.transpose(255.0 * item.clip(0,1.0)[:, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for item in y_]


    offset = [np.transpose(offset_i[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for offset_i in offset]
    filter = [np.transpose(
        filter_i[0, :, intPaddingTop:intPaddingTop + intHeight, intPaddingLeft: intPaddingLeft + intWidth],
        (1, 2, 0)) for filter_i in filter]  if filter is not None else None
    X1 = np.transpose(255.0 * X1.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))
    
    return y_[0]
    
    
def interpolate_old(im1, im2):
    X0 = torch.from_numpy( np.transpose(im1, (2,0,1)).astype('float32')/255.0).type(dtype)
    X1 = torch.from_numpy( np.transpose(im2, (2,0,1)).astype('float32')/255.0).type(dtype)

    y_ = torch.FloatTensor()

    assert (X0.size(1) == X1.size(1))
    assert (X0.size(2) == X1.size(2))
    
    intWidth = X0.size(2)
    intHeight = X0.size(1)
    channel = X0.size(0)

    if intWidth != ((intWidth >> 7) << 7):
        intWidth_pad = (((intWidth >> 7) + 1) << 7)  # more than necessary
        intPaddingLeft =int(( intWidth_pad - intWidth)/2)
        intPaddingRight = intWidth_pad - intWidth - intPaddingLeft
    else:
        intWidth_pad = intWidth
        intPaddingLeft = 32
        intPaddingRight= 32

    if intHeight != ((intHeight >> 7) << 7):
        intHeight_pad = (((intHeight >> 7) + 1) << 7)  # more than necessary
        intPaddingTop = int((intHeight_pad - intHeight) / 2)
        intPaddingBottom = intHeight_pad - intHeight - intPaddingTop
    else:
        intHeight_pad = intHeight
        intPaddingTop = 32
        intPaddingBottom = 32

    pader = torch.nn.ReplicationPad2d([intPaddingLeft, intPaddingRight , intPaddingTop, intPaddingBottom])
    
    torch.set_grad_enabled(False)
    X0 = Variable(torch.unsqueeze(X0,0))
    X1 = Variable(torch.unsqueeze(X1,0))
    X0 = pader(X0)
    X1 = pader(X1)
    
    
    if use_cuda:
        X0 = X0.cuda()
        X1 = X1.cuda()
    model.eval()
    y_s,offset,filter = model(torch.stack((X0, X1),dim = 0))
    y_ = y_s[save_which]
    
    
    if use_cuda:
        X0 = X0.data.cpu().numpy()
        if not isinstance(y_, list):
            y_ = y_.data.cpu().numpy()
        else:
            y_ = [item.data.cpu().numpy() for item in y_]
        offset = [offset_i.data.cpu().numpy() for offset_i in offset]
        filter = [filter_i.data.cpu().numpy() for filter_i in filter]  if filter[0] is not None else None
        X1 = X1.data.cpu().numpy()
    else:
        X0 = X0.data.numpy()
        if not isinstance(y_, list):
            y_ = y_.data.numpy()
        else:
            y_ = [item.data.numpy() for item in y_]
        offset = [offset_i.data.numpy() for offset_i in offset]
        filter = [filter_i.data.numpy() for filter_i in filter]
        X1 = X1.data.numpy()
        
    X0 = np.transpose(255.0 * X0.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))


    y_ = [np.transpose(255.0 * item.clip(0,1.0)[:, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for item in y_]
    # 0 weggehaald als index

    offset = [np.transpose(offset_i[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0)) for offset_i in offset]
    filter = [np.transpose(
        filter_i[0, :, intPaddingTop:intPaddingTop + intHeight, intPaddingLeft: intPaddingLeft + intWidth],
        (1, 2, 0)) for filter_i in filter]  if filter is not None else None
    X1 = np.transpose(255.0 * X1.clip(0,1.0)[0, :, intPaddingTop:intPaddingTop+intHeight, intPaddingLeft: intPaddingLeft+intWidth], (1, 2, 0))
    del X0; del X1
    return y_[0]


In [75]:
def psnr(y_hat, y):
    R = np.max([y_hat, y])
    return 10 * np.log10(R**2 / mse(y_hat, y))

def mse(y_hat, y):
    return np.mean((y_hat-y)**2)

def ssim(y_hat, y):
    None

In [53]:
interpolated.dtype

dtype('float32')

In [72]:
torch.manual_seed(0)

In [76]:
model = model.eval()
psnrs = []
ssims = []
ies   = []
for folder in tqdm(os.listdir('MiddleBurySet/other-data')):
    image1 = imread(os.path.join('MiddleBurySet/other-data', folder, 'frame10.png'))
    image2 = imread(os.path.join('MiddleBurySet/other-data', folder, 'frame11.png'))
    imagei = imread(os.path.join('MiddleBurySet/other-gt-interp', folder, 'frame10i11.png'))
    
    imagei = imagei.astype('float32')
    
    interpolated= interpolate_old(image1, image2)
    
    psnr_score = psnr(interpolated, imagei)
    ie_score = np.sqrt(mse(interpolated, imagei))
    ssim_score = compare_ssim(interpolated, imagei, multichannel=True)
    psnrs.append(psnr_score)
    ssims.append(ssim_score)
    ies.append(ie_score)
#     print(psnr_score, ssim_score)

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2494: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
..\torch\csrc\autograd\python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
c:\users\ruth\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:2693: UserWarning: Default grid_sample and affine_grid behavior will be changed to align_

In [77]:
np.mean(psnrs), np.mean(ssims), np.mean(ies)

(36.43683571482669, 0.8642908167888198, 4.1719284)

In [74]:
np.mean(psnrs), np.mean(ssims)

(36.43683515128822, 0.8642908151116094)